In [4]:
#Source: https://repl.it/@adasikow/VivaciousHollowBear

# Algorithm that generates all partitions of n-element set to k-elements subsets,
# that can be used as 'round generator' for Swiss tournaments.

# Adam Algorithm

import math
 
 
class SubsetsGenerator:
   
  def __init__(self, n, k):
    self.n = n
    self.k = k
   
   
  # debug purpose
  def get_subsets_to_assign(self, subsets_remaining):
    finish = False
    result = []
    subset_nr = 1
    while not finish and subset_nr < len(subsets_remaining):
      if subsets_remaining[subset_nr] == self.k:
        finish = True
       
      if subsets_remaining[subset_nr] > 0:
        result.append(subset_nr)
       
      subset_nr += 1
       
    return result
     
   
  def generate_debug(self, current_subsets, current_player, subsets_remaining):
    if current_player == len(current_subsets):
      print current_subsets
      return
     
    for subset_nr in self.get_subsets_to_assign(subsets_remaining):
      subsets_remaining[subset_nr] -= 1
      current_subsets[current_player] = subset_nr
      self.generate(current_subsets, current_player + 1, subsets_remaining)
      subsets_remaining[subset_nr] += 1
   
   
  def generate(self, current_subsets, current_player, subsets_remaining):
    if current_player == len(current_subsets):
      print current_subsets
      return
     
    for subset_nr in range(1, len(subsets_remaining)):
      if subsets_remaining[subset_nr] > 0:
        subsets_remaining[subset_nr] -= 1
        current_subsets[current_player] = subset_nr
        self.generate(current_subsets, current_player + 1, subsets_remaining)
        subsets_remaining[subset_nr] += 1
        if subsets_remaining[subset_nr] == self.k:
          return
     
     
def main():
  n = 6
  k = 2
  max_subsets = int(math.floor(n / k))
   
  current_subsets = [0 for i in range(0, n)]
  subsets_remaining = [k for i in range(0, max_subsets + 1)]
  subsets_remaining[0] = 0
   
  sg = SubsetsGenerator(n, k)
  sg.generate(current_subsets, 0, subsets_remaining)

main()

[1, 1, 2, 2, 3, 3]
[1, 1, 2, 3, 2, 3]
[1, 1, 2, 3, 3, 2]
[1, 2, 1, 2, 3, 3]
[1, 2, 1, 3, 2, 3]
[1, 2, 1, 3, 3, 2]
[1, 2, 2, 1, 3, 3]
[1, 2, 2, 3, 1, 3]
[1, 2, 2, 3, 3, 1]
[1, 2, 3, 1, 2, 3]
[1, 2, 3, 1, 3, 2]
[1, 2, 3, 2, 1, 3]
[1, 2, 3, 2, 3, 1]
[1, 2, 3, 3, 1, 2]
[1, 2, 3, 3, 2, 1]


In [36]:
# Seeding engine visual model

import graphviz as gv

import functools
graph = functools.partial(gv.Graph, format='svg')
digraph = functools.partial(gv.Digraph, format='svg')

def add_nodes(graph, nodes):
    for n in nodes:
        if isinstance(n, tuple):
            graph.node(n[0], **n[1])
        else:
            graph.node(n)
    return graph

def add_edges(graph, edges):
    for e in edges:
        if isinstance(e[0], tuple):
            graph.edge(*e[0], **e[1])
        else:
            graph.edge(*e)
    return graph

def apply_styles(graph, styles):
    graph.graph_attr.update(
        ('graph' in styles and styles['graph']) or {}
    )
    graph.node_attr.update(
        ('nodes' in styles and styles['nodes']) or {}
    )
    graph.edge_attr.update(
        ('edges' in styles and styles['edges']) or {}
    )
    return graph

styles = {
    'graph': {
        'label': 'Seeding Engine',
        'fontsize': '16',
        'fontcolor': 'yellow',
        'bgcolor': '#333333',
        'rankdir': 'TB',
    },
    'nodes': {
        'fontname': 'Helvetica',
        'shape': 'oval',
        'fontcolor': 'white',
        'color': 'white',
        'style': 'filled',
        'fillcolor': '#006699',
    },
    'edges': {
        'style': 'solid',
        'color': 'white',
        'arrowhead': 'open',
        'fontname': 'Courier',
        'fontsize': '12',
        'fontcolor': 'white',
    },
}

se_model = add_edges(
    add_nodes(digraph(), [
        ('A', {'label': 'Historical Match Data', 'shape': 'box', 'fillcolor': 'green'}),
        ('B', {'label': "Adam's Algorithm"}),
        ('C', {'label': 'Current Draft Data', 'shape': 'box', 'fillcolor': 'green'}),
        ('D', {'label': 'Win Ratio Builder'}),
        ('E', {'label': 'Swiss Round Builder'}),
        ('F', {'label': 'Matchmaking Engine'}),
        ('G', {'label': 'Generated Seeds', 'shape': 'box', 'fillcolor': 'orange', 'fontcolor': 'black'})
    ]),
    [
        (('C', 'F'), {'label': 'list of players'}),
        (('C', 'B'), {'label': 'number of players'}),
        (('B', 'E'), {'label': 'generated partitions'}),
        (('A', 'D'), {'label': 'match data'}),
        (('E', 'F'), {'label': 'all valid round sets'}),
        (('D', 'F'), {'label': 'win ratio matrix'}),
        (('F', 'G'), {'label': 'ordered round set'}),
        (('C', 'D'), {'label': 'list of players'}),
    ]
)

se_model = apply_styles(se_model, styles)

se_model.render('img/se_model')

'img/se_model.svg'

### Seeding Engine Specification

**INPUT I**: List of players from current tournament (and number of players)
**INPUT II**: Historical match data - all known matches from previous drafts
**OUTPUT**: Ordered set of rounds (sets of matches), that are expected to make the tournament most interesting - in every round, the sum of win % difference between players who are playing each other is minimised.

### Seeding Engine Workflow
(1) Number of players is given as input to **Adams Algorithm**. If this number is odd, then it gets incremented. **Adam Algorithm** is generating all possible partitions of the n-element set into 2-element subsets, which represents the Swiss rounds.
(2) Partitions generated in (1) are used to generate all possible round sets (there cant be two rounds with the same match, so basically every round is treated as set of player pairs). Then all round sets are verified, and the ones that have size equal to number of players - 1, are returned as output.
(3) Historical data is loaded and list of players is provided to the **Win Ratio Builder**. This part of module searches for all matches that have both players from given list, then builds a matrix that represens win rates for player vs player. If a given player pair has no results from previous matches, then they have a 50/50 result by default. Also, any player has 50/50 versus BYE player (that represents the pause round, so it won't mess the scores in any way). BYE player is added to the list if number of players is odd. Win Ratio Matrix is the output.
(4) Win Ratio matrix and all valid round sets are input to matchmaking engine. Each possible round set is evaluated by using data from win ratio, and each round set is scored, then it is ordered by score round (score is the difference between players win ratio vs each other).

